# BookRAG

## Установка библиотек

In [ ]:
!pip install langchain

In [ ]:
!pip install -U langchain-community

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install faiss-gpu

In [ ]:
!pip install openai

In [ ]:
!pip install pypdf

In [ ]:
!pip install tiktoken

## Препроцессинг данных

In [8]:
from langchain_community.document_loaders import PyPDFLoader
import re
from typing import List, Dict

In [9]:
PDF_FILE_PATH = '/kaggle/input/karamazovy/dostoevskiy_bratya_karamazovy.pdf'

loader = PyPDFLoader(PDF_FILE_PATH)
pages = loader.load()

total_pages = len(pages)
print(f"Всего страниц: {total_pages}")

# Объединение текста всех страниц в одну строку
text = ' '.join([page.page_content for page in pages])

# Убираем переносы слов (например, "припоминае-\nмого" -> "припоминаемого")
text = re.sub(r'-\n', '', text)

# Заменяем разрывы строк на пробелы (например, "\n" -> " ")
text = re.sub(r'\n', ' ', text)

# Удаляем лишние пробелы, которые могли появиться после замены
text = re.sub(r'\s+', ' ', text).strip()

Всего страниц: 510


In [10]:
def split_text_into_chunks_with_metadata(pages, chunk_size=500):
    chunks = []
    for page in pages:
        text = page.page_content
        page_number = page.metadata['page'] + 1
        current_book = None
        current_chapter = None
        current_chapter_title = None

        # Поиск текущей книги, главы и названия
        for line in text.splitlines():
            book_match = re.match(r"^\s*Книга\s+(\w+)", line)
            chapter_match = re.match(r"^\s*([IVXLCDM]+)\s*$", line)
            if book_match:
                current_book = f"Книга {book_match.group(1)}"
            if chapter_match:
                current_chapter = f"Глава {chapter_match.group(1)}"
            # Название главы на следующей строке
            chapter_title_match = re.match(r"^\s*(.*)$", line)
            if chapter_title_match and current_chapter:
                current_chapter_title = chapter_title_match.group(1)

        # Разбиваем текст на чанки
        for i in range(0, len(text), chunk_size):
            chunk = text[i:i + chunk_size]
            chunks.append({
                'chunk': chunk,
                'page': page_number,
                'book': current_book,
                'chapter': current_chapter,
                'chapter_title': current_chapter_title
            })
    return chunks

chunks = split_text_into_chunks_with_metadata(pages)
print(f"Всего чанков: {len(chunks)}")

Всего чанков: 3937


In [11]:
chunks[7]

{'chunk': 'мечателен, но решительно сомневаюсь, успею ли это доказать читателю. Дело\nв том, что это, пожалуй, и деятель, но деятель неопределенный, невыяснившийся. Впрочем,\nстранно бы требовать в такое время, как наше, от людей ясности. Одно, пожалуй, довольно\nнесомненно: это человек странный, даже чудак. Но странность и чудачество скорее вредят,\nчем дают право на внимание, особенно когда все стремятся к тому, чтоб объединить частно-\nсти и найти хоть какой-нибудь общий толк во всеобщей бестолочи. Чудак же ',
 'page': 6,
 'book': None,
 'chapter': None,
 'chapter_title': None}

## Реализация RAG

In [12]:
import faiss
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain import OpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [14]:
# Инициализация модели эмбеддингов
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

# Создание списка текстов для индексации
texts = [chunk['chunk'] for chunk in chunks]
metadatas = [chunk for chunk in chunks]

# Создание векторного хранилища FAISS
vector_store = FAISS.from_texts(texts, embeddings, metadatas=metadatas)

# Сохранение индекса (опционально)
vector_store.save_local("faiss_index")

/tmp/ipykernel_30/475268992.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=api_key)


In [15]:
# Инициализация модели генерации ответов
llm = OpenAI(
    temperature=0.3,  # Низкая креативность для точных ответов
    openai_api_key=api_key,
    max_tokens=1024  # Увеличьте значение по необходимости
)

/tmp/ipykernel_30/3517174068.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(


In [16]:
template = """
Вы являетесь ассистентом, который отвечает на вопросы по книге "Братья Карамазовы" Фёдора Достоевского.
Используйте предоставленные отрывки из книги для формирования ответов. 
Обязательно указывай краткие цитаты и указывайте книгу, главу и название главы, из которой была взята информация.
Перепроверь, что в промпте указан развернутый ответ на вопрос, глава и цитата книги, использованные для ответа. Если не указана, обязательно укажи, используя метаинформацию.

Контекст:
{context}

Вопрос: {question}

Ответ:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

In [17]:
# Создание цепочки Вопрос-Ответ с использованием RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Использование простого объединения
    retriever=vector_store.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
# Функция Чат-Бота
def chatbot():
    print("Чат-бот по книге 'Братья Карамазовы'. Задайте свой вопрос или введите 'выход' для завершения.")
    while True:
        try:
            user_input = input("Вы: ")
            if user_input.lower() in ['выход', 'exit', 'quit']:
                print("Чат-бот: До свидания!")
                break
            response = qa_chain.invoke(user_input)["result"]
            print(f"Чат-бот: {response}")
        except Exception as e:
            print(f"Чат-бот: Извините, произошла ошибка: {e}")

if __name__ == "__main__":
    chatbot()

Чат-бот по книге 'Братья Карамазовы'. Задайте свой вопрос или введите 'выход' для завершения.


Вы:  кто главный долбоеб книги


Чат-бот: Главный герой книги "Братья Карамазовы" - Дмитрий Карамазов. В главе "Записки о детстве" он говорит: "Ненавидят дурное, а про себя все его любят". Он также признается, что читает "дурные книги" и крадет их у своей мамы. В главе "Записки о Тихоне" Петр Ильич называет Дмитрия "голубчиком" и приглашает выпить с ним в бакалейном магазине. В главе "Записки о Тихоне" Дмитрий объясняет свою деятельность как "жизнеописание" и признается, что у него есть два романа, один из которых произошел еще в его юности. Таким образом, Дмитрий Карамазов является главным героем и долбоебом книги.


Вы:  кто главные персонажи книги


Чат-бот: Главными персонажами книги "Братья Карамазовы" являются братья Алексей, Дмитрий и Иван Карамазовы, а также их отец Федор Павлович Карамазов. Это подтверждается в главе "Пролог. Беседа о беседе" где автор пишет: "Итак, к рассказу. Когда еще до свету положили уготованное к погребению тело старца во гроб и вынесли его в первую, бывшую приемную комнату, то возник было между находившимися у гроба вопрос: надо ли о таким-то и таким-то сделкам, в которые сам тогда-то и тогда пожелал вступить, он и права не имеет требовать ничего более, и проч., и проч." Также в главе "Книга первая. Беспокойный сон" автор упоминает о трех сыновьях Федора Павловича: "Но, пока перейду к этому роману, нужно еще рассказать и об остальных двух сыновьях Федора Павловича".


...

Вы:  чем братья карамазовы лучше, чем книга идиот


Чат-бот: 
В главе 119 герой Алеша замечает, что братья Карамазовы стали добрее, чем раньше. Он говорит: "Вот вы теперь и добрее стали". Это показывает, что герой считает, что братья Карамазовы стали лучше, чем были раньше. Он также добавляет: "Я тебя и без коньяку люблю", что означает, что он любит своего брата Алешу не за то, что он пьет, а просто за то, что он есть. Это показывает, что братья Карамазовы лучше, чем книга "Идиот", где герой принимает людей такими, какие они есть, без осуждения и предубеждений.

В главе 422 Иван говорит о своих мыслях и признает, что они скверные и глупые. Он говорит: "Только всё скверные мои мысли берешь, а главное – глупые". Это показывает, что Иван осознает свои ошибки и старается быть лучше. Это делает его лучше, чем герой книги "Идиот", который не признает свои ошибки и не стремится к самосовершенствованию.

В главе 446 герой Митя говорит о своем брате Иване и его уме. Он говорит: "Один ум хорошо, а два гораздо лучше". Это показывает, что Митя цени

Вы:  в чем завязка сюжета книги


Чат-бот: Завязка сюжета книги "Братья Карамазовы" заключается в том, что молодой человек, будущий герой Алеша, узнает о мошенничестве своего отца Федора Павловича и его желании вступить в сделки, которые могут привести к катастрофе. Это становится переломным моментом в жизни Алеши, который в дальнейшем будет стремиться к раскрытию правды и раскаянию за свои поступки. Это событие также влияет на других героев книги, в том числе на Ивана, который впоследствии станет одним из главных противников отца и будет мучиться раскаянием за измену своей возлюбленной Мите.


Вы:  в какой главе умирает федор
